# API dati.gov.it

In [70]:
# import libraries
import pandas as pd
import numpy as np

## Lista dei Dataset Pubblicati sul sito dati.gov.it

In [34]:
# Recupera la lista degli Open Data
url_list = 'http://www.dati.gov.it/api/3/action/package_list'
df_list = pd.read_json(url_list)
# Numero di dataset
df_list['result'].count()

17829

## Check Metadati del Dataset

In [142]:
df_list_test = df_list[0:10]

In [135]:
def check_metadato_singolo(df_name):
    url_meta = 'http://www.dati.gov.it/api/3/action/package_show?id='+df_name
    try:
        df_meta = pd.read_json(url_meta)
        check = True
    # Cattura eccezione Http 404, manca il metadato
    except:
        check = False
    return check

In [150]:
def check_metadati_all(df_list):
    l = []
    for index, row in df_list.iterrows():
        d = {}
        d['name']=row['result']
        d['check_metadati']=check_metadato_singolo(row['result'])
        l.append(d)
    df = pd.DataFrame.from_dict(l)
    return df

In [151]:
df = check_metadati_all(df_list_test)

In [152]:
# Se False vuol dire che i Metadati sono assenti
df.head()

,check_metadati,name
0,False,circoscrizioni
1,True,elezioni-amministrative-2014-eletti
2,True,censimento-pillole-2013
3,True,rilevazione-dei-prezzi-consumo
4,True,suoli-non-recintati


## Get Metadati

In [153]:
# Se metadato presente allora lo recupero
def get_metadati(df_name):
    url_meta = 'http://www.dati.gov.it/api/3/action/package_show?id='+df_name
    try:
        df_meta = pd.read_json(url_meta)
        metadati = df_meta['result']
    # Cattura eccezione Http 404, manca il metadato
    except:
        metadati = np.nan
    return metadati

In [154]:
def get_resource(metadato):
    res = np.nan
    if 'resources' in metadato:
        if(metadato['resources'][-1]['mimetype']=='csv'):
            res = metadato['resources'][-1]
    return res

In [155]:
s = get_metadati(df_list['result'][7])
r = get_resource(s)
r

{'created': '',
 'description': '',
 'format': 'csv',
 'id': '1f259a9c-7ddd-4383-954a-8370a063c189',
 'last_modified': '',
 'mimetype': 'csv',
 'name': '2013-02-13T140538/organico.csv',
 'resource_group_id': '3dfbcaa7-ff9a-4853-b041-54310ea4b73e',
 'revision_id': '',
 'revision_timestamp': 'Mercoledì 12 Aprile 2017',
 'size': '',
 'state': 'Active',
 'url': 'http://opendata.comune.bari.it/dataset/e334d4a1-ef8a-412f-8431-d5c68c74e88f/resource/9a75b69a-f3a8-4f8b-ba8b-f8c4cf465fdb/download/20130213t140538organico.csv'}

## Get Dataset from Url

In [158]:
df = pd.read_csv(r['url'])

In [159]:
df.head(2)

,cognome,nome,UFFICIO,PROFILO
0,ABBATANTUONO,VITO,3^Circoscrizione,Dirigente Amministrativo
1,ABBATICCHIO,ROBERTO,Segreteria Generale-Informazioni,Esecutore Informatore-Notificatore
